In [1]:
import sqlite3
import pandas as pd

In [7]:
#Taskas:
#example

In [2]:
# Connect to or create a new database file called 'superstore_database.db'

conn = sqlite3.connect('superstore_database.db')

In [3]:
# Create a cursor

cursor = conn.cursor()

In [4]:
# Perform a query

cursor.execute("SELECT * FROM Customers WHERE CustomerType = 'Corporate'")
rows = cursor.fetchall()

In [5]:
rows

[('DV-13045', 'Darrin Van Huff', 'Corporate'),
 ('KB-16585', 'Ken Black', 'Corporate'),
 ('GH-14485', 'Gene Hale', 'Corporate'),
 ('LC-16930', 'Linda Cazamias', 'Corporate'),
 ('RA-19885', 'Ruben Ausman', 'Corporate'),
 ('ES-14080', 'Erin Smith', 'Corporate'),
 ('ON-18715', 'Odella Nelson', 'Corporate'),
 ('JM-15265', 'Janet Molinari', 'Corporate'),
 ('BS-11590', 'Brendan Sweed', 'Corporate'),
 ('KB-16600', 'Ken Brennan', 'Corporate'),
 ('SC-20770', 'Stewart Carmichael', 'Corporate'),
 ('JC-16105', 'Julie Creighton', 'Corporate'),
 ('JS-15685', 'Jim Sink', 'Corporate'),
 ('ER-13855', 'Elpida Rittenbach', 'Corporate'),
 ('CV-12805', 'Cynthia Voltz', 'Corporate'),
 ('GG-14650', 'Greg Guthrie', 'Corporate'),
 ('DW-13585', 'Dorothy Wardle', 'Corporate'),
 ('JD-15895', 'Jonathan Doherty', 'Corporate'),
 ('SH-19975', 'Sally Hughsby', 'Corporate'),
 ('JE-16165', 'Justin Ellison', 'Corporate'),
 ('SP-20650', 'Stephanie Phelps', 'Corporate'),
 ('JB-15400', 'Jennifer Braxton', 'Corporate'),
 ('D

In [6]:
# Convert the retrieved data into a Dataframe
Corporate_customers = pd.DataFrame([row[1:3] for row in rows], columns= ['Customer ID', 'Customer Name'])

In [ ]:
#1. What is the category generating the maximum sales revenue?

# What about the profit in this category?
# Are they making a loss in any categories?

In [11]:
query = """
WITH one AS (
SELECT *
FROM OrderDetails O JOIN Products P
ON O.ProductID=P.ProductID)

SELECT Category, SUM(ProductPrice * Quantity) AS TotalSales
FROM one
GROUP BY Category
ORDER BY TotalSales DESC
LIMIT 1;
"""

# Execute the query
cursor.execute(query)
max_sales_category = cursor.fetchone()
print(f"The category generating the maximum sales revenue is {max_sales_category[0]} with a revenue of ${max_sales_category[1]:.2f}.")


The category generating the maximum sales revenue is Technology with a revenue of $101459234.00.


In [17]:
# This SQL query calculates total sales and total profit for each category
query = """
WITH one AS (
SELECT *
FROM OrderDetails O JOIN Products P
ON O.ProductID=P.ProductID)

SELECT Category, 
       SUM(ProductPrice *(1-Discount)* Quantity) AS TotalSales,
       SUM((ProductPrice * (1-Discount) - SupplierPrice) * Quantity) AS TotalProfit
FROM one
GROUP BY Category
ORDER BY TotalSales DESC
LIMIT 1;
"""

# Execute the query
cursor.execute(query)
result = cursor.fetchone()
category, total_sales, total_profit = result  # Unpack the result

print(f"The category with the highest sales revenue is '{category}' with a revenue of ${total_sales:.2f}.")
print(f"The total profit for this category is ${total_profit:.2f}.")


The category with the highest sales revenue is 'Technology' with a revenue of $81478926.40.
The total profit for this category is $14105653.40.


In [16]:
# SQL query to calculate total profit for each category including the effect of the discount
# Assuming 'Discount' column represents the discount as a fraction of the price (e.g., 0.1 for 10%)
query = """
WITH one AS (
SELECT *
FROM OrderDetails O JOIN Products P
ON O.ProductID=P.ProductID)

SELECT Category, 
       SUM(((ProductPrice * (1 - Discount)) - SupplierPrice) * Quantity) AS TotalProfit
FROM one
GROUP BY Category
HAVING TotalProfit < 0;
"""

# Execute the query
cursor.execute(query)
loss_making_categories = cursor.fetchall()

# Check if any categories are making a loss
if loss_making_categories:
    print("Categories making a loss (considering discounts):")
    for category, total_profit in loss_making_categories:
        print(f"{category}: Loss of ${-total_profit:.2f}")
else:
    print("No categories are making a loss considering discounts.")
    print("No categories are making a loss.")

No categories are making a loss considering discounts.
No categories are making a loss.


In [30]:
# What are 5 states generating the maximum and minimum sales revenue?

In [33]:
# SQL query to calculate total sales revenue considering product price and discount
query = """
WITH Revenue AS (
    SELECT
        Address.State AS State,
        SUM(Products.ProductPrice * (1 - OrderDetails.Discount) * OrderDetails.Quantity) AS TotalRevenue
    FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    JOIN Shipment ON OrderDetails.SalesID = Shipment.SalesID
    JOIN Address ON Shipment.AddressID = Address.AddressID
    GROUP BY Address.State
)
SELECT State, TotalRevenue
FROM Revenue
ORDER BY TotalRevenue DESC
LIMIT 5;
"""

try:
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()

    # Print the results
    print("Top 5 States by Maximum Sales Revenue:")
    for state, revenue in results:
        print(f"{state}: ${revenue:.2f}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")



Top 5 States by Maximum Sales Revenue:
California: $89970547.70
New York: $62087984.00
Texas: $33994341.84
Washington: $27053444.40
Pennsylvania: $22960233.40


In [34]:
query = """
WITH Revenue AS (
    SELECT
        Address.State AS State,
        SUM(Products.ProductPrice * (1 - OrderDetails.Discount) * OrderDetails.Quantity) AS TotalRevenue
    FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    JOIN Shipment ON OrderDetails.SalesID = Shipment.SalesID
    JOIN Address ON Shipment.AddressID = Address.AddressID
    GROUP BY Address.State
)
SELECT State, TotalRevenue
FROM Revenue
ORDER BY TotalRevenue ASC
LIMIT 5;
"""

try:
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()

    # Print the results
    print("Top 5 States by Min Sales Revenue:")
    for state, revenue in results:
        print(f"{state}: ${revenue:.2f}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

Top 5 States by Min Sales Revenue:
West Virginia: $172044.80
North Dakota: $183982.00
Maine: $254100.00
South Dakota: $263112.00
Wyoming: $320627.20


In [ ]:
#What are the 3 products in each product segment with the highest sales?


In [36]:
# SQL query to find the top 3 products in each category by sales revenue
query = """

SELECT Category, ProductName, SalesRevenue
FROM (
    SELECT
        Products.Category,
        Products.ProductName,
        SUM(Products.ProductPrice * (1 - OrderDetails.Discount) * OrderDetails.Quantity) AS SalesRevenue,
        RANK() OVER (PARTITION BY Products.Category ORDER BY SUM(Products.ProductPrice * (1 - OrderDetails.Discount) * OrderDetails.Quantity) DESC) as Rank
    FROM Products
    JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
    GROUP BY Products.Category, Products.ProductName
)
WHERE Rank <= 3;
"""

try:
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()

    # Print the results
    print("Top 3 Products by Sales Revenue in Each Category:")
    for category, product_name, sales_revenue in results:
        print(f"Category: {category}, Product: {product_name}, Sales Revenue: ${sales_revenue:.2f}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

Top 3 Products by Sales Revenue in Each Category:
Category: Furniture, Product: HON 5400 Series Task Chairs for Big and Tall, Sales Revenue: $2187057.60
Category: Furniture, Product: DMI Eclipse Executive Suite Bookcases, Sales Revenue: $1770964.30
Category: Furniture, Product: Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish, Sales Revenue: $1561096.56
Category: Office Supplies, Product: Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind, Sales Revenue: $2745338.40
Category: Office Supplies, Product: GBC DocuBind TL300 Electric Binding System, Sales Revenue: $1982347.90
Category: Office Supplies, Product: GBC Ibimaster 500 Manual ProClick Binding System, Sales Revenue: $1902450.00
Category: Technology, Product: Canon imageCLASS 2200 Advanced Copier, Sales Revenue: $6159982.40
Category: Technology, Product: Cisco TelePresence System EX90 Videoconferencing Unit, Sales Revenue: $2263848.00
Category: Technology, Product: Hewlett Packard LaserJet 3310 

In [ ]:
#Are they the 3 most profitable products as well?

In [39]:
# SQL query to calculate and rank products by both sales revenue and profit
query = """
WITH RevenueProfit AS (
    SELECT
        Products.Category,
        Products.ProductName,
        SUM(Products.ProductPrice * (1 - OrderDetails.Discount) * OrderDetails.Quantity) AS SalesRevenue,
        SUM((Products.ProductPrice * (1 - OrderDetails.Discount) - Products.SupplierPrice) * OrderDetails.Quantity) AS Profit,
        RANK() OVER (PARTITION BY Products.Category ORDER BY SUM(Products.ProductPrice * (1 - OrderDetails.Discount) * OrderDetails.Quantity) DESC) AS RevenueRank,
        RANK() OVER (PARTITION BY Products.Category ORDER BY SUM((Products.ProductPrice * (1 - OrderDetails.Discount) - Products.SupplierPrice) * OrderDetails.Quantity) DESC) AS ProfitRank
    FROM Products
    JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
    GROUP BY Products.Category, Products.ProductName
)
SELECT 
    Category, 
    ProductName, 
    SalesRevenue, 
    Profit, 
    RevenueRank, 
    ProfitRank
FROM RevenueProfit
WHERE RevenueRank <= 3 OR ProfitRank <= 3;
"""

try:
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()

    # Print the results
    print("Products ranked top 3 by Sales Revenue or Profit in Each Category:")
    for category, product_name, sales_revenue, profit, revenue_rank, profit_rank in results:
        print(f"Category: {category}, Product: {product_name}, Sales Revenue: ${sales_revenue:.2f}, Profit: ${profit:.2f}, Revenue Rank: {revenue_rank}, Profit Rank: {profit_rank}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

Products ranked top 3 by Sales Revenue or Profit in Each Category:
Category: Furniture, Product: HON 5400 Series Task Chairs for Big and Tall, Sales Revenue: $2187057.60, Profit: $15.60, Revenue Rank: 1, Profit Rank: 250
Category: Furniture, Product: DMI Eclipse Executive Suite Bookcases, Sales Revenue: $1770964.30, Profit: $66646.30, Revenue Rank: 2, Profit Rank: 25
Category: Furniture, Product: Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish, Sales Revenue: $1561096.56, Profit: $-66943.44, Revenue Rank: 3, Profit Rank: 359
Category: Furniture, Product: Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back, Sales Revenue: $1063752.80, Profit: $192723.80, Revenue Rank: 7, Profit Rank: 1
Category: Furniture, Product: Global Deluxe High-Back Manager's Chair, Sales Revenue: $866519.40, Profit: $155844.40, Revenue Rank: 12, Profit Rank: 2
Category: Furniture, Product: Hon Pagoda Stacking Chairs, Sales Revenue: $648379.60, Profit: $154061.60, Revenue Rank: 26, Profit Ran

In [42]:
#What are the 3 best-seller products in each product segment? (Quantity-wise)
# SQL query to find the top 3 best-selling products in each segment by quantity
query = """
WITH QuantityRanking AS (
    SELECT
        Products.Category,
        Products.ProductName,
        SUM(OrderDetails.Quantity) AS TotalQuantity,
        RANK() OVER (PARTITION BY Products.Category ORDER BY SUM(OrderDetails.Quantity) DESC) as Rank
    FROM Products
    JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
    GROUP BY Products.Category, Products.ProductName
)
SELECT 
    Category, 
    ProductName, 
    TotalQuantity, 
    Rank
FROM QuantityRanking
WHERE Rank <= 3;
"""

try:
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()

    # Print the results
    print("Top 3 Best-Selling Products by Quantity in Each Segment:")
    for segment, product_name, total_quantity, rank in results:
        print(f"Category: {segment}, Product: {product_name}, Total Quantity Sold: {total_quantity}, Rank: {rank}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

Top 3 Best-Selling Products by Quantity in Each Segment:
Category: Furniture, Product: KI Adjustable-Height Table, Total Quantity Sold: 74, Rank: 1
Category: Furniture, Product: Situations Contoured Folding Chairs, 4/Set, Total Quantity Sold: 64, Rank: 2
Category: Furniture, Product: Staple-based wall hangings, Total Quantity Sold: 62, Rank: 3
Category: Office Supplies, Product: Staples, Total Quantity Sold: 215, Rank: 1
Category: Office Supplies, Product: Staple envelope, Total Quantity Sold: 170, Rank: 2
Category: Office Supplies, Product: Easy-staple paper, Total Quantity Sold: 150, Rank: 3
Category: Technology, Product: Imation 16GB Mini TravelDrive USB 2.0 Flash Drive, Total Quantity Sold: 75, Rank: 1
Category: Technology, Product: Logitech G19 Programmable Gaming Keyboard, Total Quantity Sold: 60, Rank: 2
Category: Technology, Product: Kingston Digital DataTraveler 16GB USB 2.0, Total Quantity Sold: 57, Rank: 3


In [43]:
# What are the top 3 worst-selling products in every category? (Quantity-wise)

In [44]:
query = """
WITH QuantityRanking AS (
    SELECT
        Products.Category,
        Products.ProductName,
        SUM(OrderDetails.Quantity) AS TotalQuantity,
        RANK() OVER (PARTITION BY Products.Category ORDER BY SUM(OrderDetails.Quantity) ASC) as Rank
    FROM Products
    JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
    GROUP BY Products.Category, Products.ProductName
)
SELECT 
    Category, 
    ProductName, 
    TotalQuantity, 
    Rank
FROM QuantityRanking
WHERE Rank <= 3;
"""

try:
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()

    # Print the results
    print("Top 3 Worst-Selling Products by Quantity in Each Category:")
    for segment, product_name, total_quantity, rank in results:
        print(f"Category: {segment}, Product: {product_name}, Total Quantity Sold: {total_quantity}, Rank: {rank}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

Top 3 Worst-Selling Products by Quantity in Each Category:
Category: Furniture, Product: Bush Saratoga Collection 5-Shelf Bookcase, Hanover Cherry, *Special Order, Total Quantity Sold: 1, Rank: 1
Category: Furniture, Product: Global Enterprise Series Seating Low-Back Swivel/Tilt Chairs, Total Quantity Sold: 1, Rank: 1
Category: Furniture, Product: Barricks Non-Folding Utility Table with Steel Legs, Laminate Tops, Total Quantity Sold: 2, Rank: 3
Category: Furniture, Product: Ultra Commercial Grade Dual Valve Door Closer, Total Quantity Sold: 2, Rank: 3
Category: Office Supplies, Product: Boston 1900 Electric Pencil Sharpener, Total Quantity Sold: 1, Rank: 1
Category: Office Supplies, Product: Xerox 20, Total Quantity Sold: 1, Rank: 1
Category: Office Supplies, Product: Avery 5, Total Quantity Sold: 2, Rank: 3
Category: Office Supplies, Product: Avery Hi-Liter Pen Style Six-Color Fluorescent Set, Total Quantity Sold: 2, Rank: 3
Category: Office Supplies, Product: Eureka Disposable Bags f

In [45]:
# How many unique customers per month are there for the year 2016. 
#(There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH() 
#to extract the year or the month in a date. You will have to create two new columns: year and month.)

In [48]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('superstore_database.db')
cursor = conn.cursor()

# Add Year and Month columns to the Orders table
try:
    cursor.execute("ALTER TABLE OrderDetails ADD COLUMN Year INTEGER;")
    cursor.execute("ALTER TABLE OrderDetails ADD COLUMN Month INTEGER;")
except sqlite3.OperationalError as e:
    print(e)  # Handle the error if the columns already exist

# Update the Year and Month columns based on the OrderDate
update_query = """
UPDATE OrderDetails
SET
    Year = CAST(SUBSTR(OrderDate, 1, 4) AS INTEGER),
    Month = CAST(SUBSTR(OrderDate, 6, 2) AS INTEGER);
"""
cursor.execute(update_query)

# Commit the changes to the database
conn.commit()

In [50]:


# Query to find unique customers per month in 2016
query = """
SELECT Year, Month, COUNT(DISTINCT CustomerID) AS UniqueCustomers
 FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    JOIN Shipment ON OrderDetails.SalesID = Shipment.SalesID
    JOIN Address ON Shipment.AddressID = Address.AddressID
WHERE Year = 2016
GROUP BY Year, Month
ORDER BY Month;
"""

cursor.execute(query)
results = cursor.fetchall()

# Print the results
print("Unique Customers per Month in 2016:")
for year, month, unique_customers in results:
    print(f"Year: {year}, Month: {month}, Unique Customers: {unique_customers}")


Unique Customers per Month in 2016:
Year: 2016, Month: 1, Unique Customers: 46
Year: 2016, Month: 2, Unique Customers: 42
Year: 2016, Month: 3, Unique Customers: 80
Year: 2016, Month: 4, Unique Customers: 83
Year: 2016, Month: 5, Unique Customers: 96
Year: 2016, Month: 6, Unique Customers: 90
Year: 2016, Month: 7, Unique Customers: 89
Year: 2016, Month: 8, Unique Customers: 86
Year: 2016, Month: 9, Unique Customers: 176
Year: 2016, Month: 10, Unique Customers: 95
Year: 2016, Month: 11, Unique Customers: 165
Year: 2016, Month: 12, Unique Customers: 157
